# How to combine split ontologies


In [1]:
from ontopy import World
world = World()
# emmo = world.get_ontology('emmo-inferred').load()
# print(f"Ontology IRI: {emmo.base_iri}")

# Canonical ontology IRI
ONTO_IRI = "http://example.org/owl-client-relationship"

# Get (or create) the ontology object once
rel = world.get_ontology(ONTO_IRI)

# Load both physical files into the SAME ontology
rel.load(filename="../relationship.ttl", format="turtle", only_local=True)

# Add the individuals file (second load would be skipped because rel.loaded is True)
rel.load(filename="../client-eg1.ttl", format="turtle", only_local=True)

# Rebuild Python wrappers so .individuals() sees new resources
rel.sync_attributes()

# rel.load(filename="../client-eg1.ttl", format="turtle", only_local=True)

print("Objects in world having same IRI reference is rel:", rel is world.get_ontology(ONTO_IRI))

print(f"rel.iri: {rel.iri}")
print(f"rel.base_iri: {rel.base_iri}")

print("\nCounts:")
print("  Classes:", len(list(rel.classes())))
print("  Individuals:", len(list(rel.individuals())))

print("\nSample individuals:", [i.name for i in list(rel.individuals())[:20]])

# If you still see 0, force reload (rarely needed)
# rel.world._props_loaded = False
# rel.reload()

# Use rel everywhere instead of relationship / instances
relationship = rel
instances = rel
# ...existing code...

Objects in world having same IRI reference is rel: True
rel.iri: None
rel.base_iri: http://example.org/owl-client-relationship#

Counts:
  Classes: 6
  Individuals: 2

Sample individuals: ['Bitovi', 'MCorp']


In [2]:
# Helper to append triples from an extra Turtle file into an existing ontology
import rdflib

def append_turtle(onto, path):
    g2 = rdflib.Graph()
    g2.parse(path, format="turtle")
    with onto:
        store_graph = onto.world.as_rdflib_graph()
        for t in g2:
            store_graph.add(t)
    onto.sync_attributes()

from ontopy import World
world = World()
rel = world.get_ontology(ONTO_IRI).load(filename="../relationship.ttl", format="turtle", only_local=True)
append_turtle(rel, "../client-eg1.ttl")
print("Individuals:", [i.name for i in rel.individuals()])

Individuals: ['Bitovi', 'MCorp', 'Adrian', 'Magesh', 'Terry', 'Travis', 'Avinav', 'Ben', 'Wade', 'Adisa']


In [3]:
from ontopy import World
world = World()

TBOX_IRI = "http://example.org/owl-client-relationship"
ABOX_IRI = "http://example.org/owl-client-relationship/abox"

world = World()

# 1. Preload TBox (classes/properties)
tbox = world.get_ontology(TBOX_IRI).load(
    filename="../relationship.ttl",
    format="turtle",
    only_local=True
)

# 2. Load ABox (individuals) that imports the TBox
abox = world.get_ontology(ABOX_IRI).load(
    filename="../client-eg1.ttl",
    format="turtle",
    only_local=True
)

print("TBox classes:", len(list(world.get_ontology("http://example.org/owl-client-relationship").classes())))
print("All individuals (ABox):", [i.name for i in abox.individuals()])
relationship = world.get_ontology("http://example.org/owl-client-relationship")
instances = abox

print(list(world.classes()))

TBox classes: 6
All individuals (ABox): ['Adrian', 'Wade', 'Adisa', 'Magesh', 'Terry', 'Travis', 'Avinav', 'Ben']
[owl-client-relationship.Organization, owl-client-relationship.Person, owl-client-relationship.Consultant, owl-client-relationship.Consultant Manager, owl-client-relationship.Client Manager, owl-client-relationship.Client]
